In [1]:
import sys
from pathlib import Path
import json

import numpy as np
import pennylane as qml
from matplotlib import pyplot as plt
from qiskit import QuantumCircuit
from qiskit.utils import QuantumInstance
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit.providers.aer import StatevectorSimulator
from qiskit.circuit.library import ZZFeatureMap
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.tensorboard import SummaryWriter
from pandas import DataFrame

sys.path.extend(['/home/quic/QUIC-Projects/'])
from classifiers.quantum.qml.qasvm import PseudoTensorSoftQASVM
from classifiers.datasets.sklearn import SklearnDataset
from classifiers.quantum.ansatz import MultilayerCircuit9FeatureMap
from classifiers.convex.svm import CvxSoftQASVM

# Grad fn simulation testing

In [11]:
n_qubits = 13
n_feature = 10

device:qml.Device = qml.device('lightning.gpu', wires=n_qubits) # TODO: gpu vs cpu
def var_form(params):
    qml.BasicEntanglerLayers(params, wires=device.wires, rotation=qml.RY)

feature_map = ZZFeatureMap(feature_dimension=n_feature, reps=2, entanglement='linear')
data = np.random.rand(2**n_qubits, n_feature)
label = np.random.randint(0, 2, 2**n_qubits)
qasvm = PseudoTensorSoftQASVM(data=data @ data.T, label=label, device=device, feature_map=None, var_form=var_form)
parameter_shape = qml.BasicEntanglerLayers.shape(n_layers=20, n_wires=device.num_wires)

grad_result = np.empty(shape=(10, *parameter_shape))
for t in range(10):
    params=qml.numpy.random.random((parameter_shape), requires_grad=True)
    grad_result[t, :, :] = qasvm.grad_fn(params)

GPU time: 2m 2.6s

In [13]:
n_qubits = 13
n_feature = 10

device:qml.Device = qml.device('lightning.qubit', wires=n_qubits) # TODO: gpu vs cpu
def var_form(params):
    qml.BasicEntanglerLayers(params, wires=device.wires, rotation=qml.RY)

feature_map = ZZFeatureMap(feature_dimension=n_feature, reps=2, entanglement='linear')
data = np.random.rand(2**n_qubits, n_feature)
label = np.random.randint(0, 2, 2**n_qubits)
qasvm = PseudoTensorSoftQASVM(data=data @ data.T, label=label, device=device, feature_map=None, var_form=var_form)
parameter_shape = qml.BasicEntanglerLayers.shape(n_layers=20, n_wires=device.num_wires)

grad_result = np.empty(shape=(10, *parameter_shape))
for t in range(10):
    params=qml.numpy.random.random((parameter_shape), requires_grad=True)
    grad_result[t, :, :] = qasvm.grad_fn(params)

CPU time: 1m 30.4s

In [14]:
n_qubits = 13
n_feature = 10

device:qml.Device = qml.device('lightning.qubit', wires=n_qubits) # TODO: gpu vs cpu
def var_form(params):
    qml.BasicEntanglerLayers(params, wires=device.wires, rotation=qml.RY)

feature_map = ZZFeatureMap(feature_dimension=n_feature, reps=2, entanglement='linear')
data = np.random.rand(2**n_qubits, n_feature)
label = np.random.randint(0, 2, 2**n_qubits)
qasvm = PseudoTensorSoftQASVM(data=data @ data.T, label=label, device=device, feature_map=None, var_form=var_form)
parameter_shape = qml.BasicEntanglerLayers.shape(n_layers=20, n_wires=device.num_wires)

grad_result = np.empty(shape=(10, *parameter_shape))
grad_fn = qml.jacobian(qasvm.cost_fn)
for t in range(10):
    params=qml.numpy.random.random((parameter_shape), requires_grad=True)
    grad_result[t, :, :] = grad_fn(params)

Exterior grad fn: 1m 29.2s

In [15]:
n_qubits = 13
n_feature = 10

device:qml.Device = qml.device('lightning.qubit', wires=n_qubits) # TODO: gpu vs cpu
def var_form(params):
    qml.BasicEntanglerLayers(params, wires=device.wires, rotation=qml.RY)

feature_map = ZZFeatureMap(feature_dimension=n_feature, reps=2, entanglement='linear')
data = np.random.rand(2**n_qubits, n_feature)
label = np.random.randint(0, 2, 2**n_qubits)
qasvm = PseudoTensorSoftQASVM(data=data @ data.T, label=label, device=device, feature_map=None, var_form=var_form)
parameter_shape = qml.BasicEntanglerLayers.shape(n_layers=20, n_wires=device.num_wires)

grad_result = np.empty(shape=(10, *parameter_shape))
grad_fn = qml.grad(qasvm.cost_fn)
for t in range(10):
    params=qml.numpy.random.random((parameter_shape), requires_grad=True)
    grad_result[t, :, :] = grad_fn(params)

`qml.jacobian` -> `qml.grad`: 1m 29.2s

In [33]:
grad_result[:, 0, 0].var()

0.004942012038085211